<a href="https://colab.research.google.com/github/HeqingZ/FCOStry/blob/main/FCOS%E6%9C%80%E6%8E%A5%E8%BF%91%E7%9A%84%E4%B8%80%E6%AC%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#mount on google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#install detectron2
!python -m pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached PyYAML-5.1-cp37-cp37m-linux_x86_64.whl
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-4m9g4v0a
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-4m9g4v0a
  Resolved https://github.com/facebookresearch/detectron2.git to commit 717ab9f0aeca216a2f800e43d705766251ba3a55
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an is

In [3]:
#import some important utilities of Detectron2
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
torch:  1.12 ; cuda:  cu113
detectron2: 0.6


In [4]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
#import features for the bbox
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

from detectron2.structures import BoxMode

In [6]:
import os
from pathlib import Path
from ast import literal_eval

import numpy as np
import pandas as pd
import cv2
import torch
import matplotlib.pyplot as plt
from PIL import Image

from detectron2.data import (DatasetCatalog, 
                             MetadataCatalog, 
                             build_detection_test_loader
                            )
from detectron2.data.datasets.coco import convert_to_coco_json
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.structures import BoxMode
from detectron2.config import get_cfg
from detectron2.utils.logger import setup_logger
from detectron2.engine import DefaultTrainer, default_setup, hooks
from detectron2.modeling import GeneralizedRCNNWithTTA
from detectron2.evaluation import COCOEvaluator, DatasetEvaluators, inference_on_dataset
 

logger = setup_logger()

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [7]:
!conda --version

conda 4.14.0


In [8]:
#fully install fcos
!conda create --name FCOS
!conda activate FCOS

Solving environment: / done

## Package Plan ##

  environment location: /usr/local/envs/FCOS



Preparing transaction: \ done
Verifying transaction: / done
Executing transaction: \ done
#
# To activate this environment, use
#
#     $ conda activate FCOS
#
# To deactivate an active environment, use
#
#     $ conda deactivate

Retrieving notices: ...working... done

CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [9]:
# we give the instructions for CUDA 9.0
!conda install -c pytorch torchvision=0.2.1 cudatoolkit=9.0


Solving environment: - failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - cudatoolkit=9.0

Current channels:

  - https://conda.anaconda.org/pytorch/linux-64
  - https://conda.anaconda.org/pytorch/noarch
  - https://conda.anaconda.org/conda-forge/linux-64
  - https://conda.anaconda.org/conda-forge/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [11]:
!export INSTALL_DIR=$PWD


In [12]:
# install pycocotools. Please make sure you have installed cython.
%cd $INSTALL_DIR
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!python setup.py build_ext install



[Errno 2] No such file or directory: '$INSTALL_DIR'
/content
Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 13.31 MiB/s, done.
Resolving deltas: 100% (576/576), done.
/content/cocoapi/PythonAPI
Traceback (most recent call last):
  File "setup.py", line 2, in <module>
    import numpy as np
ModuleNotFoundError: No module named 'numpy'


In [13]:
# install PyTorch Detection
%cd $INSTALL_DIR
!git clone https://github.com/tianzhi0549/FCOS.git
%cd FCOS



[Errno 2] No such file or directory: '$INSTALL_DIR'
/content/cocoapi/PythonAPI
Cloning into 'FCOS'...
remote: Enumerating objects: 2102, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 2102 (delta 6), reused 10 (delta 5), pack-reused 2090
Receiving objects: 100% (2102/2102), 8.81 MiB | 11.77 MiB/s, done.
Resolving deltas: 100% (1370/1370), done.
/content/cocoapi/PythonAPI/FCOS


In [31]:

import torch
import numpy as np

In [16]:
# the following will install the lib with
# symbolic links, so that you can modify
# the files if you want and won't need to
# re-build it
!python setup.py build develop --no-deps


!unset INSTALL_DIR


Traceback (most recent call last):
  File "setup.py", line 7, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'


In [17]:
#define the file path to retrieve images
def FLIR(source_path, img_dir, anno_dir ): #define a function. assuming user doesnt know shit about coding and enter random stuff. related to anno dir and img dir
    img_dir = os.path.join(source_path, img_dir)
    anno_dir = os.path.join(source_path, anno_dir)
    data_dicts = []
    #define annotation properties
    with open(anno_dir) as F:
      jsonfile = json.load(F) #load annotations from json file into the dictionary
    
    annotationinfo = jsonfile["annotations"]
    imagesinfo = jsonfile["images"] #retreieve info like category ID and bbox
   
    #define img sources
     
    img_dir_list = os.listdir(img_dir)#get the list of all files and directories in the specified directory.
    # print(img_dir)
    for img_info in imagesinfo: #bbox info category id
      for images in img_dir_list:  #img_dir_list = img_dir, which is source_path/img_dir(original)
        images = "data/" + images
        if images == img_info['file_name']: #see if name matches each other, because we wanna find the filenames only for the small flir out of the whole json file

          record={}
          filename = os.path.join(source_path, img_info["file_name"])
          # print(filename)
          record["file_name"] = filename
          record["image_id"] = img_info['id']
          record["height"] = img_info['height']
          record['width'] = img_info['width']
        
          box_info = []
          
          for anno in annotationinfo:
            if anno["image_id"] == img_info['id']:
              # if (anno['category_id'] <= 3) & (anno['category_id'] > 0):
              if (anno['category_id'] == 1) & (anno['category_id'] > 0): # Predict people only
                        box = {
                        "bbox": anno['bbox'],
                        'bbox_mode': BoxMode.XYWH_ABS, # Category_id starts from 0 in detectron2
                        'category_id': int(anno['category_id']-1)  #we forcibly convert this shit into integer to be able to minus 1. just incase the stuff from the dict is integer
                        #computer counts from zero but not hyman. we have to substract one to match the computer #update Jun29: i deleted -1, bcz there is no category 0 
                        } #exactly how dictionary looks like
                        imagetesthaha = cv2.imread(filename, 0) 
                        tt = cv2.rectangle(imagetesthaha, anno["bbox"], (255,0,0),2)
                        #cv2_imshow(tt) #for printing all boxes in images one by one, will crash
                        box_info.append(box)
          record["annotations"] = box_info

          data_dicts.append(record)
    return data_dicts

      
      
    #print(type(jsonfile)) #check the  type of jsonfile. dict or list or stuff



#define dataloader for training and testing sets
#train_dataloader = dataloader(training_data, batch_size=64, shuffle=True)
#test_dataloader = dataloader(test_data, batch_size=64, shuffle=True)


In [18]:
def FLIR_dataset_wrapper(source_path, img_dir, anno_dir):
    def wrapper():
        return FLIR(source_path, img_dir, anno_dir)
    return wrapper

In [19]:
DatasetCatalog.clear() #unregister stuff

In [20]:
for d in ["train", "val"]:
    DatasetCatalog.register("FLIR_THERMAL_" + d + "_data", lambda d=d: FLIR("/content/drive/MyDrive/FLIR_THERMAL/train", "data", "coco.json")) #changed "/directry/stuff/" +d +"shit" into this rn
    MetadataCatalog.get("FLIR_THERMAL_" + d + "_data").set(thing_classes=["FLIR_THERMAL"])
FLIR_metadata = MetadataCatalog.get("FLIR_THERMAL_train_data")


In [21]:
convert_to_coco_json('FLIR_THERMAL_train_data', output_file='content/drive/MyDrive/output/inference/FLIR_Thermal_Train_coco_format.json', allow_cached=False)


[08/31 07:01:14 d2.data.datasets.coco]: Converting annotations of dataset 'FLIR_THERMAL_train_data' to COCO format ...)
[08/31 07:17:05 d2.data.datasets.coco]: Converting dataset dicts into COCO format
[08/31 07:17:10 d2.data.datasets.coco]: Conversion finished, #images: 10742, #annotations: 50478
[08/31 07:17:10 d2.data.datasets.coco]: Caching COCO format annotations at 'content/drive/MyDrive/output/inference/FLIR_Thermal_Train_coco_format.json' ...


In [22]:
#to varify our thing by calling the function
dataset_dicts = FLIR("/content/drive/MyDrive/FLIR_THERMAL/train", "data", "coco.json")


In [23]:
!wget https://cloudstor.aarnet.edu.au/plus/s/glqFc13cCoEyHYy/download -O fcos_R_50_1x.pth #download pretrained model

--2022-08-31 07:28:37--  https://cloudstor.aarnet.edu.au/plus/s/glqFc13cCoEyHYy/download
Resolving cloudstor.aarnet.edu.au (cloudstor.aarnet.edu.au)... 202.158.207.20
Connecting to cloudstor.aarnet.edu.au (cloudstor.aarnet.edu.au)|202.158.207.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Syntax error in Set-Cookie: 5230042dc1897=giapqbut8sl7jfs3at0ghb1nvi; path=/plus;; Secure at position 53.
Syntax error in Set-Cookie: oc_sessionPassphrase=a%2F4AXm%2F8wqYSnyuZ5xnXVdMYTXBdOpJaYIA9hcED%2F5ZniJa8TYAucPnuKeG5sF2f564q7KhdNeSKF2Wg1mRQ1S8wekoZT2qk25k9T4yxCvgmxAqAKT9x8DRI%2FrN9ptXs; expires=Thu, 01-Sep-2022 07:28:38 GMT; Max-Age=86400; path=/plus;; Secure at position 224.
Length: 129471951 (123M) [application/octet-stream]
Saving to: ‘fcos_R_50_1x.pth’

fcos_R_50_1x.pth    100%[===================>] 123.47M  10.9MB/s    in 13s     

2022-08-31 07:28:51 (9.44 MB/s) - ‘fcos_R_50_1x.pth’ saved [129471951/129471951]



TRAIN！

In [ ]:
'''
before training we have to change the file in  /content/cocoapi/PythonAPI/FCOS/fcos_core/config/paths_catalog.py

        "FLIR_THERMAL_train_data":{
            "img_dir": "/content/drive/MyDrive/FLIR_THERMAL/train/data",
            "ann_file": "/content/drive/MyDrive/FLIR_THERMAL/train/coco.json"          
        },
       "FLIR_THERMAL_val_data":{
            "img_dir": "/content/drive/MyDrive/FLIR_THERMAL/val/data",
            "ann_file": "/content/drive/MyDrive/FLIR_THERMAL/val/coco.json"          
        },
add these 2 datasets manually


'''

In [ ]:
from detectron2.engine import DefaultTrainer


cfg = get_cfg()
cfg.DATASETS.TRAIN = ("FLIR_THERMAL_train_data",)
cfg.MODEL.RESNETS.DEPTH = 50 #for mvit v2 S we are setting the depth = 101. (F)
# cfg.DATASETS.TEST = ('SMALL_FLIR_THERMAL_val_data',)

!python -m torch.distributed.launch \
    --nproc_per_node=8 \
    --master_port=$((RANDOM + 10000)) \
    tools/train_net.py \
    --skip-test \
    --config-file /content/cocoapi/PythonAPI/FCOS/fcos/configs/fcos_R_50_FPN_1x.yaml \
    DATALOADER.NUM_WORKERS 2 \
    OUTPUT_DIR /content/drive/MyDrive/fcos_R_50_FPN_1x

# cfg.merge_from_file("/content/swin/configs/SwinT/faster_rcnn_swint_T_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = "/content/cocoapi/PythonAPI/FCOS/fcos_R_50_1x.pth" 
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 4  # This is the real "batch size" commonly known to deep learning people 'changed to 5'
cfg.SOLVER.BASE_LR = 0.0001 #SET ALL TO 0.0001
cfg.SOLVER.MAX_ITER = 270000    # 270000 for 3x (F)
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.SOLVER.WARMUP_FACTOR = 1.0 / 5000
cfg.SOLVER.WARMUP_ITERS = 5000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # chose 512 for 3x
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (people). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.MODEL.MASK_ON = False
# cfg.MODEL.RESNETS.DEFORM_MODULATED = True
cfg.MODEL.BACKBONE.FREEZE_AT = 0
cfg.OUTPUT_DIR = "/content/drive/MyDrive/FCOSTEST"





# # load pretrained weights
# other_weights = torch.load('/content/swin_tiny_patch4_window7_224_d2.pth')['model']
# cfg.self_weight = trainer.model.state_dict()
# # for name, param in self_weight.items():
# #     if name in other_weights:
# #         if other_weights[name].shape == param.shape:
# #             self_weight[name] = other_weights[name]
# #         else:
# #             print(f"size mismatch at {name}")
# #     else:
# #         print(f"layer {name} does not exist")
# cfg.trainer.model.load_state_dict(self_weight)


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=True)
trainer.train()

/usr/local/bin/python: Error while finding module specification for 'torch.distributed.launch' (ModuleNotFoundError: No module named 'torch')
[08/31 07:36:55 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3